# Major notes and ideas:

- make unecessery model diagnostics or computed values as on_demand variables instead, this improves efficiency (if they do not have to be calculated at every step)

- add index variable functionality! but how?

-

In [1]:
import xso

In [2]:
# start testing base functionality
xso.variable()

xso.component()

<function xso.component.component.<locals>.create_component(cls)>

In [3]:
import xsimlab as xs

In [4]:
xs.process()

<function xsimlab.process.process.<locals>.wrap(cls)>

In [5]:
import xsimlab as xs

from xso.core import XSOCore


@xs.process
class Backend:
    """this object contains the backend model and is modified or read by all other components"""

    solver_type = xs.variable(intent='in')
    m = xs.any_object(description='model backend instance is stored here')

    def initialize(self):
        print('initializing model backend')
        self.m = PhydraCore(self.solver_type)

    def finalize(self):
        print('finalizing: cleanup')
        self.m.cleanup()  # for now only affects gekko solve

In [6]:
Backend

__main__.Backend

# Now Test The Global Ref option

- test the feature, if is incorporated into XSO, now is the time!
- go through and update the framework, then build Phydra with new funcs

In [7]:
@xs.process
class Globally:
    glo = xs.global_ref(name="A")
    testgroup = xs.group_dict(name="G")
    
    def initialize(self):
        print(self.glo)
        print(self.testgroup)
        
    def finalize(self):
        print("finalize globally")
        print(self.testgroup)

In [8]:
@xs.process
class Varr:
    a = xs.variable(global_name="A")
    
    b = xs.variable(groups="G")
    c = xs.variable(groups="G")
    
    def run(self):
        print(self.a)
        self.a = 2
        print(self.a)
        
    def finalize(self):
        print("finalize Varr")
        print(self.a)

In [9]:
testmod = xs.Model({'g':Globally, 'var':Varr})

In [10]:
test = xs.create_setup(model=testmod, 
                       clocks={'time':[0,1]},
                       input_vars={'var':{'a':1, 'b':'B', 'c':3}})

In [11]:
with testmod:
    test.xsimlab.run()

1
Frozen({('var', 'b'): 'B', ('var', 'c'): 3})
finalize globally
Frozen({('var', 'b'): 'B', ('var', 'c'): 3})
finalize Varr
1


# Should I build it from "scratch"?

- or start modifying, and then go through and trim?

seems like the second option is more efficient, can always do a full rewrite part per part

hm... now, where to start?

build "test" xso components, then go through backend and modify, whilst testing basic functionality

In [12]:
@xso.component(init_stage=2)
class Variable:
    var = xso.variable(description='basic state variable')
    
@xso.component(init_stage=2)
class Variable_with_dims:
    var = xso.variable(dims='var', description='basic state variable')
    
@xso.component(init_stage=3)
class Flux:
    """ basic flux process for testing 
    represents linear growth of a foreign variable
    """
    var = xso.variable(foreign=True, flux='growth', 
                       negative=False, description='variable affected by flux',
                      dims=[(),('var')])
    rate = xso.parameter(description='linear rate of change',
                      dims=[(),('var')])

    @xso.flux(dims=[(),('var')])
    def growth(self, var, rate):
        """ """
        return var * rate

TypeError: 'NoneType' object is not iterable

In [ ]:
MODEL = xso.create({"Var":Variable, "VarDims":Variable_with_dims, "Flux":Flux, "Flux_dims":Flux})

In [13]:
MODEL

NameError: name 'MODEL' is not defined

In [16]:
import numpy as np

SETUP = xso.setup(solver='odeint', model=MODEL,
            time=np.arange(0,5),
            input_vars={'Var':{'var_label':'A', 'var_init':1.},
                        'VarDims':{'var_label':'B', 'var_init':[1.,2.]},
                        'Flux':{'var':'A', 'rate':1.},
                        'Flux_dims':{'var':'B', 'rate':1.},
                       }
         )

In [17]:
with MODEL:
    OUTPUT = SETUP.xsimlab.run()

initializing model backend
Initializing Model Time
Initializing component Var
Initializing component VarDims
Initializing component Flux
Initializing component Flux_dims
assembling model
SOLVER : <xso.solvers.ODEINTSolver object at 0x7f6026362230>
Model is assembled:
Model contains: 
Variables:['time', 'A', 'B'] 
Parameters:['Flux_rate', 'Flux_dims_rate'] 
Forcings:[] 
Fluxes:['Time_time_flux', 'Flux_growth', 'Flux_dims_growth'] 
Full Model Dimensions:[('time', None), ('A', None), ('B', 2), ('Time_time_flux', None), ('Flux_growth', None), ('Flux_dims_growth', 2)] 

finalizing: cleanup
Model was solved in 0.02174 seconds


ValueError: Output array of 1 dimension(s) for variable 'Flux__growth_value' doesn't match any of its accepted dimension(s): (((), 'var', 'time'),)

In [55]:
OUTPUT

<xarray.Dataset>
Dimensions:             (time: 5, input_time: 5, var: 2, clock: 2)
Coordinates:
  * clock               (clock) int64 0 1
Dimensions without coordinates: time, input_time, var
Data variables:
    Core__solver_type   <U6 'odeint'
    Flux__growth_value  (time) float64 1.718 1.718 4.671 12.7 34.51
    Flux__rate          float64 1.0
    Flux__var           <U1 'A'
    Time__time          (input_time) int64 0 1 2 3 4
    VarDims__var_init   (var) float64 1.0 2.0
    VarDims__var_label  <U1 'B'
    VarDims__var_value  (var, time) float64 1.0 1.0 1.0 1.0 ... 2.0 2.0 2.0 2.0
    Var__var_init       float64 1.0
    Var__var_label      <U1 'A'
    Var__var_value      (time) float64 1.0 2.718 7.389 20.09 54.6